In [120]:
PATTERN_CURRENT_STAKING_DOLLARS = r"\$\d{1,3}(,\d{3})*(\.\d{2})?"
PATTERN_EBC_STAKING = r"\d{1,3}(,\d{3})*(\.\d+)?(?=\sEBC)"
PATTERN_HOLDERS = r"\d{1,3}(,\d{3})*"
PATTERN_EBC_VALUE = r"\$\d+(\.\d{1,4})?"
PATTERN_MARKETCAP = r"\$\d{1,3}(,\d{3})*(\.\d{2})?"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
STAKING_URL = f"https://polygonscan.com/address/0xDEb03f15098921989Bd554395a4E6745Fc2755E4"
TOTALS_URL = f"https://polygonscan.com/token/0x3f7ca3e85305aed836636999d054519f6ccc4056"

In [121]:
from bs4 import BeautifulSoup
import requests
import re

In [122]:
def fetch_page(url, headers, retries=3):
    """Tenta buscar a página com retries."""
    for _ in range(retries):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response
    return None

In [123]:
def extract_staking_value(html):
    """Extrai o valor em dólares do staking usando BeautifulSoup e regex."""
    soup = BeautifulSoup(html, "html.parser")
    button = soup.find("button", {"id": "dropdownMenuBalance"})
    if not button:
        raise ValueError("Elemento com ID 'dropdownMenuBalance' não encontrado.")
    
    text = button.text
    match = re.search(PATTERN_CURRENT_STAKING_DOLLARS, text)
    if not match:
        raise ValueError("Valor em dólares não encontrado no texto do botão.")
    
    # Converte o valor para float
    value = float(match.group(0).strip("$").replace(",", ""))
    return value

def get_current_staking_dollars():
    """Função principal para obter o valor de staking."""
    response = fetch_page(STAKING_URL, HEADERS)
    if not response:
        raise ConnectionError("Falha ao obter a página após múltiplas tentativas.")
    
    try:
        value = extract_staking_value(response.content)
        return value
    except Exception as e:
        print(f"Erro ao extrair valor de staking: {e}")
        return None

In [124]:
def extract_ebc_staking(html):
    """Extrai a quantidade de EBC em staking."""
    soup = BeautifulSoup(html, "html.parser")
    # Procura o <span> que contém "EBC"
    staking_span = soup.find("span", class_="text-muted", string=re.compile(r"EBC"))
    if not staking_span:
        raise ValueError("Elemento contendo a quantidade de EBC em staking não encontrado.")
    
    # Usa regex para capturar o número antes de "EBC"
    match = re.search(PATTERN_EBC_STAKING, staking_span.text)
    if not match:
        raise ValueError("Valor de EBC não encontrado no texto do elemento.")
    
    # Remove vírgulas e converte para float
    staking_value = float(match.group(0).replace(",", ""))
    return staking_value

def get_ebc_staking_quantity():
    """Função principal para obter a quantidade de EBC em staking."""
    response = fetch_page(STAKING_URL, HEADERS)
    if not response:
        raise ConnectionError("Falha ao obter a página após múltiplas tentativas.")
    
    try:
        value = extract_ebc_staking(response.content)
        return value
    except Exception as e:
        print(f"Erro ao extrair quantidade de EBC em staking: {e}")
        return None

In [125]:
def extract_holders(html):
    """Extrai o total de holders."""
    soup = BeautifulSoup(html, "html.parser")
    # Encontra o elemento que contém os holders
    holders_div = soup.find("div", class_="d-flex flex-wrap gap-2")
    if not holders_div:
        raise ValueError("Elemento contendo o total de holders não encontrado.")
    
    # Usa regex para capturar o número
    match = re.search(PATTERN_HOLDERS, holders_div.text)
    if not match:
        raise ValueError("Total de holders não encontrado.")
    
    # Remove vírgulas e converte para inteiro
    holders = int(match.group(0).replace(",", ""))
    return holders

def get_holders():
    """Função principal para obter o total de holders."""
    response = fetch_page(TOTALS_URL, HEADERS)
    if not response:
        raise ConnectionError("Falha ao obter a página após múltiplas tentativas.")
    
    try:
        holders = extract_holders(response.content)
        return holders
    except Exception as e:
        print(f"Erro ao extrair total de holders: {e}")
        return None

In [126]:
def extract_ebc_value(html):
    """Extrai o valor do EBC em dólares."""
    soup = BeautifulSoup(html, "html.parser")
    
    # Localiza a div específica pelo ID
    value_div = soup.find("div", id="ContentPlaceHolder1_tr_valuepertoken")
    if not value_div:
        raise ValueError("Div com ID 'ContentPlaceHolder1_tr_valuepertoken' não encontrada.")
    
    # Busca o span que contém o valor do preço do EBC
    value_span = value_div.find("span", {"data-bs-toggle": "tooltip", "data-bs-html": "true"})
    if not value_span:
        raise ValueError("Elemento contendo o valor do EBC não encontrado dentro da div.")
    
    # Extrai o valor do texto
    value_text = value_span.text.strip("$")
    try:
        ebc_value = float(value_text)
        return ebc_value
    except ValueError:
        raise ValueError(f"Valor '{value_text}' não é um número válido.")

def get_ebc_value():
    """Função principal para obter o valor do EBC em dólares."""
    response = fetch_page(TOTALS_URL, HEADERS)
    if not response:
        raise ConnectionError("Falha ao obter a página após múltiplas tentativas.")
    
    try:
        value = extract_ebc_value(response.content)
        return value
    except Exception as e:
        print(f"Erro ao extrair valor do EBC: {e}")
        return None

In [127]:
def extract_marketcap(html):
    """Extrai o valor do Market Cap."""
    soup = BeautifulSoup(html, "html.parser")
    
    # Localiza a div pelo ID específico
    marketcap_div = soup.find("div", id="ContentPlaceHolder1_tr_marketcap")
    if not marketcap_div:
        raise ValueError("Div com ID 'ContentPlaceHolder1_tr_marketcap' não encontrada.")
    
    # Extrai o texto do valor do Market Cap
    marketcap_text = marketcap_div.text
    match = re.search(PATTERN_MARKETCAP, marketcap_text)
    if not match:
        raise ValueError("Valor do Market Cap não encontrado no texto da div.")
    
    # Remove o símbolo $ e as vírgulas, depois converte para float
    marketcap_value = float(match.group(0).replace(",", "").strip("$"))
    return marketcap_value

def get_marketcap():
    """Função principal para obter o Market Cap."""
    response = fetch_page(TOTALS_URL, HEADERS)
    if not response:
        raise ConnectionError("Falha ao obter a página após múltiplas tentativas.")
    
    try:
        value = extract_marketcap(response.content)
        return value
    except Exception as e:
        print(f"Erro ao extrair Market Cap: {e}")
        return None

In [128]:
get_current_staking_dollars()

15696342.15

In [129]:
get_ebc_staking_quantity()

64919122.0

In [130]:
get_holders()

19619

In [131]:
get_ebc_value()

0.2418

In [132]:
get_marketcap()

331242753.7